In [153]:
import pandas as pd
key = "2022"
path = f"./asset/新建文件夹_2024_第二版/副本{key}.xlsx"
outpath = f"./output/{key}.xlsx"
# 读取Excel文件
df_pipei = pd.read_excel(path,sheet_name="Sheet1")
df_zcpm = pd.read_excel(path,sheet_name="Sheet2")

# def findCloest(bankName,df,blackBankNameSet):
#     """
#     找到离 bankName 资产水平最近的银行
#     """
#     # print(f"closet bank to {bankName}")
#     target_row = df.loc[df["id"]==bankName, "资产"].values;
#     # 如果target_row为空, 那就返回空 series
#     if len(target_row)==0:
#         print(f"没有找到银行 {bankName} 的资产")
#         return pd.Series()
#     data_zs= target_row[0]

#     # df=df[df["银行"]!=bankName]
#     df=df[~df['id'].isin(blackBankNameSet)]
#     # # 创建一个 DataFrame
#     # # 计算招商银行资产与其他银行资产的差
#     diff = df["资产"] - data_zs

#     # # 找到与招商银行资产最相近的银行
#     closest_bank = df.loc[diff.abs().idxmin()]
#     return closest_bank


# 在 df_pipei 里找到和 bankCode相同的 bankName
def findBlankWithSameCode(bankCode):
    """
    在 df_pipei 里找到和 bankCode相同的 bankName
    """
    df_pipei_copy =df_pipei.copy();
    df_pipei_copy = df_pipei_copy[df_pipei_copy["证券代码"]==bankCode]
    # print(f"和{bankCode}相同的银行有：\n{df_pipei_copy}")
    return df_pipei_copy["贷款银行"]

# findBlankWithSameCode(2030)

# 获取对应的银行的资产
def getBankInfo(bankName,df):
    """
    获取银行资产
    """
    target_row = df.loc[df["id"]==bankName].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        return pd.Series()
    return target_row[0]
def exportExcel(df):
    """
    导出数据到Excel
    """
    df.to_excel(outpath, sheet_name=key)


# 从 df_pipei 找到 bankName 匹配的行
def findByBlankNameInPipei(bankName):
    target_row = df_pipei.loc[df_pipei["贷款银行"]==bankName].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        return pd.Series()
    return target_row[0]


df_pipei_copy =df_pipei.copy();

In [154]:
def findCloest(bankName,bankType,df,blackBankNameSet):
    target_row = df.loc[df["id"]==bankName, "资产"].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        print(f"没有找到银行 {bankName} 的资产")
        exit
        # return pd.Series()
    data_zs= target_row[0]

    # df=df[df["银行"]!=bankName]
    df=df[~df['id'].isin(blackBankNameSet)]

    # # 计算目标银行资产与其他银行资产的差
    diff = df["资产"] - data_zs
        # # 找到与招商银行资产最相近的银行
    closest_bank = df.loc[diff.abs().idxmin()]


    df_insameType = df[(df["类型"]==bankType) & (df["资产"]>=data_zs-1000) & (df["资产"]<=data_zs+1000)]
    # df_insameType = df[(df["类型"]==bankType)]
    if( len(df_insameType)>=1):
        # print(f"df_insameType:\n{df_insameType}")
        diff_insampType = df_insameType["资产"] - data_zs
        closest_bank=df_insameType.loc[diff_insampType.abs().idxmin()]
    else:
        # print(f"同类型({bankType})中找不到匹配的银行")    
        pass

    # print(f"closest_bank:\n {closest_bank}")
    return closest_bank



In [155]:
df_pipei_copy = []
df_pipei_copy =df_pipei.copy();

# iters= df_pipei.iloc[:10].iterrows()
iters= df_pipei.iterrows()
for index,row in iters:
    bankName = row["贷款银行"]
    bankCode = row["证券代码"]
    # 如果bankName是空串或者bankCod 是空串 那就 continue
    if bankName=="" or bankCode=="":
        continue
    # print(bankName:bankName)
    row_bankinfo = getBankInfo(bankName,df_zcpm)
    # print(f"row_bankType:{row_bankinfo}")
    if len(row_bankinfo)<1:
        print(f"没有在资产配置表里找到{bankName}对应的银行")
        continue
    row_bankType=row_bankinfo[2]
    res= findCloest(bankName,row_bankType,df_zcpm,findBlankWithSameCode(bankCode))
    print
    if(len(res)==0):
        print(f"找不到{bankName}最近的元素")
        continue
    _name=res["id"]
    # print(f"{bankName}对应的银行:{_name}")
    df_pipei_copy.loc[index,"当前银行资产"]=row_bankinfo[1]
    df_pipei_copy.loc[index,"当前银行类型"]=row_bankinfo[2]
    #修改 df_pipei_copy 的 index 行为 [...rowres[0]res[1]]
    df_pipei_copy.loc[index,"最近银行"]=_name
    df_pipei_copy.loc[index,"最近银行资产"]=res["资产"]
    df_pipei_copy.loc[index,"最近银行类型"]=res["类型"]
    info = findByBlankNameInPipei(_name)
    if len(info)==0:
        print(f"在配置表里 没有找到银行 [{_name}] 的信息")
        continue
    # print(info)
    df_pipei_copy.loc["index最近银行代码"]= info[2]
print(f"前十名 {df_pipei_copy[:10]}")
exportExcel(df_pipei_copy)


没有在资产配置表里找到大连银行对应的银行
在配置表里 没有找到银行 [威海银行] 的信息
在配置表里 没有找到银行 [石嘴山银行] 的信息
在配置表里 没有找到银行 [石嘴山银行] 的信息
在配置表里 没有找到银行 [威海银行] 的信息
在配置表里 没有找到银行 [石嘴山银行] 的信息
在配置表里 没有找到银行 [九台农商银行] 的信息
在配置表里 没有找到银行 [陕西秦农农村商业银行] 的信息
在配置表里 没有找到银行 [陕西秦农农村商业银行] 的信息
在配置表里 没有找到银行 [陕西秦农农村商业银行] 的信息
在配置表里 没有找到银行 [陕西秦农农村商业银行] 的信息
在配置表里 没有找到银行 [乐山市商业银行] 的信息
在配置表里 没有找到银行 [金华银行] 的信息
在配置表里 没有找到银行 [肥西农村商业银行] 的信息
在配置表里 没有找到银行 [枣庄银行] 的信息
在配置表里 没有找到银行 [肥西农村商业银行] 的信息
在配置表里 没有找到银行 [江苏溧水农村商业银行] 的信息
没有在资产配置表里找到大连银行对应的银行
在配置表里 没有找到银行 [重庆农村商业银行] 的信息
在配置表里 没有找到银行 [重庆农村商业银行] 的信息
在配置表里 没有找到银行 [浙江泰隆商业银行] 的信息
没有在资产配置表里找到大连银行对应的银行
没有在资产配置表里找到大连银行对应的银行
在配置表里 没有找到银行 [保定银行] 的信息
在配置表里 没有找到银行 [陕西秦农农村商业银行] 的信息
在配置表里 没有找到银行 [九台农商银行] 的信息
在配置表里 没有找到银行 [江苏南通农村商业银行] 的信息
在配置表里 没有找到银行 [江苏南通农村商业银行] 的信息
没有在资产配置表里找到江苏启东农村商业银行对应的银行
没有在资产配置表里找到江苏启东农村商业银行对应的银行
没有在资产配置表里找到江苏启东农村商业银行对应的银行
在配置表里 没有找到银行 [广东南海农村商业银行] 的信息
在配置表里 没有找到银行 [温州银行] 的信息
没有在资产配置表里找到银行对应的银行
在配置表里 没有找到银行 [浙江泰隆商业银行] 的信息
在配置表里 没有找到银行 [温州银行] 的信息
没有在资产配置表里找到大连银行对应的银行
没有在资产配置表里找到大连银行对应的银行
在配置表里 没有找到银行 [